1️⃣ Secrets & API-key bootstrap

In [ ]:
# ────────────────────────────────────────────────────────────────────────
#  SECRETS → LOCAL FILES  |  runs before any other import
# ────────────────────────────────────────────────────────────────────────
import os, base64, pathlib, json
import openai
from openai import OpenAI

def _decode_and_validate(secret_name: str, out_file: str):
    """Decode Base64 GitHub secret and validate JSON structure."""
    try:
        decoded = base64.b64decode(os.environ[secret_name]).decode('utf-8')
        data = json.loads(decoded)

        # For token.json, verify required fields
        if out_file == "token.json":
            required_fields = {'token', 'refresh_token', 'scopes'}
            if not required_fields.issubset(data.keys()):
                missing = required_fields - set(data.keys())
                raise ValueError(f"Missing required fields in token: {missing}")

        pathlib.Path(out_file).write_text(decoded)
        return True
    except Exception as e:
        print(f"❌ Error processing {secret_name}: {str(e)}")
        raise

try:
    _decode_and_validate("CLIENT_SECRETS_JSON", "client_secrets.json")
    _decode_and_validate("TOKEN_JSON", "token.json")
except Exception as e:
    print("❌ Failed to initialize secrets")
    raise

openai.api_key = os.environ["OPENAI_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]  # <-- add this line
client = OpenAI()
print("✅ Secrets decoded and validated, OpenAI client ready.")


2️⃣ Parameter cell

In [ ]:
# PARAMETERS  (Papermill / run-notebook overwrites these at run-time)
GAMEPLAY_URL = None                         # GitHub secret injects real link
TITLE_TEXT   = "Daily DALLE Short"          # you can override this too
OPENAI_MODEL = "gpt-4o"


🔐 3. Authenticate with YouTube

In [ ]:
# 🔐 2. Authenticate with YouTube (automatic – no browser needed)
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json

# Use only the essential scope needed for uploads
SCOPE = "https://www.googleapis.com/auth/youtube.upload"

def get_authenticated_service():
    """Create authenticated YouTube client with scope validation."""
    try:
        # Load token data
        with open("token.json") as f:
            token_data = json.load(f)

        # Verify the token has our required scope
        if 'scopes' not in token_data or SCOPE not in token_data['scopes']:
            raise ValueError(f"Token missing required scope: {SCOPE}")

        creds = Credentials.from_authorized_user_info(token_data, [SCOPE])

        # Refresh token if needed
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
                # Update token file with refreshed credentials
                with open("token.json", "w") as f:
                    json.dump(json.loads(creds.to_json()), f)
            except Exception as refresh_error:
                print(f"⚠️ Token refresh failed: {refresh_error}")
                # Continue with expired token if we have one
                if not creds.token:
                    raise

        return build("youtube", "v3", credentials=creds)
    except HttpError as e:
        print(f"❌ YouTube API error: {e}")
        raise
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        raise

try:
    youtube = get_authenticated_service()
    print(f"✅ YouTube API authenticated with scope: {SCOPE}")
except Exception as e:
    print("❌ Failed to initialize YouTube client")
    raise

4-7: THIS IS WHERE WE BUILD OUR SHORTS DRAMA STORIES GENERATION CODE BLOCKS

1️⃣ Generate a Unique Story Seed (Topic)

In [ ]:
# ───────────────────────────────────────────────────────────────
# 1️⃣ Generate a unique seed/topic for each short story run from Reddit r/WritingPrompts

import random, time, re
import praw

def generate_story_seed():
    # 1) Initialize PRAW Reddit client (use your credentials or environment variables)
    reddit = praw.Reddit(
        client_id="IPF2FtGAO8sLhzTUbEGSsQ",
        client_secret="RLO_IjGT9qIjnaYPxlVctFF3XL4Cjg",
        user_agent="ShortStoryBot by /u/Rexxus25"
    )

    # 2) Fetch top submissions from r/WritingPrompts in the past 48 hours
    sub = reddit.subreddit("WritingPrompts")
    now = time.time()
    two_days_ago = now - 48 * 3600

    # Grab the top 50 from the past week, filter to last 48 h, sort by score
    posts = list(sub.top(time_filter="week", limit=50))
    recent = [p for p in posts if p.created_utc >= two_days_ago and not p.stickied]
    recent_sorted = sorted(recent, key=lambda p: p.score, reverse=True)
    top20 = recent_sorted[:20] if len(recent_sorted) >= 20 else recent_sorted

    # If not enough recent, fallback to whatever we have
    if not top20:
        top20 = posts[:20]

    # 3) Clean up the prompt: remove any [XYZ] at the start, e.g. [WP], [PI], etc.
    def clean_title(title):
        return re.sub(r"^\[[^\]]+\]\s*", "", title).strip()

    # 4) Pick one at random and clean it
    selected = random.choice(top20)
    seed = clean_title(selected.title)

    print(f"🔄 Auto‑selected story seed: {seed}")
    return seed

STORY_SEED = generate_story_seed()


2️⃣ Generate a Short Story Script Using GPT-4o-mini

In [ ]:
# ───────────────────────────────────────────────────────────────
# 2️⃣ Generate a viral short story script (<50 seconds, high retention) with GPT-4o-mini, plus descriptions for consistency

from openai import OpenAI

def generate_short_story(seed, example_story=None):
    system_prompt = (
        "You are a viral short story writer for YouTube. "
        "Write a story designed for narration in under 40 seconds. "
        "The story must be no more than 130 words. "
        "In the very first sentence, clearly establish the setting and main character in just a few words before the action starts, as to answer the audiences question of why am i watching this. "
        "Start immediately with a dramatic moment, conflict, or shocking discovery. "
        "Use stronghuman emotional triggers, something like betrayal, revenge, sacrifice, secrets, danger, or loss. "
        "Keep the pace brisk; every sentence should advance the drama. "
        "Include at some dialogue, but mainly storytell. "
        "End with a twist, punchline, or moment of realization. This is important and should give the clear message and lesson learned and leave the audience with some food for thought "
        "Conclude with a clear final line that delivers the story’s lesson, message, or a thought-provoking statement that makes the audience wonder about the situation or story told. "
        "Make the audience feel why they watched the story by having this clear conclusion or ending message or lessen learned from the story. "
        "Make sure the audience understands the who/where/when within the first 2 sentences before plunging into the conflict. "
        "Be direct and vivid. No introduction or explanation—write only the story. "
        "Limit the story to 130 words maximum."
    )
    few_shot_example = (
        example_story or
    """
At midnight, in the rain-soaked alley behind the city’s last jazz club, Mia clutched her broken saxophone.

A stranger’s shadow fell across her, voice cold: “Play, or you’ll never see your brother again.”

Mia’s trembling fingers obeyed, music mingling with sirens as the night spiraled into chaos.

The masked man leaned close. “You think you’re the only one who lost something tonight?”

Only later, as dawn broke and the alley was empty, did Mia realize—sometimes you must lose your song to find your voice.

Trust the night, but never trust its promises.
    """
    )

    prompt = (
        f"{system_prompt}\n"
        f"Example structure:\n{few_shot_example}\n\n"
        f"Now, write a unique and engaging story based on this topic:\n\"{seed}\""
    )

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=510,
        temperature=1,
    )
    story_script = response.choices[0].message.content.strip()
    print(f"Generated short story:\n{story_script}")

    # Main character appearance (as before)
    char_desc_prompt = (
        "Based on the following short story, describe the main character's appearance (age, gender, hair, eyes, clothing, skin color, facial characteristics other notable features) in one vivid sentence. "
        "Make up plausible details if they are not specified. Do NOT describe their personality, only looks. "
        "Output only the appearance description, no extra text.\n\n"
        f"Story:\n{story_script}\n"
    )
    char_resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": char_desc_prompt}],
        max_tokens=50,
        temperature=0.7,
    )
    main_character_description = char_resp.choices[0].message.content.strip().replace('"', '')

    # Supporting characters
    supp_char_prompt = (
        "Based on the following short story, briefly describe up to two other important characters if present (name, age, gender, appearance, clothing, skin color). "
        "Output a single sentence for each, or write 'None' if no supporting characters are clearly present.\n\n"
        f"Story:\n{story_script}\n"
    )
    supp_char_resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": supp_char_prompt}],
        max_tokens=90,
        temperature=0.7,
    )
    supporting_characters_description = supp_char_resp.choices[0].message.content.strip().replace('"', '')

    # Setting/environment
    env_prompt = (
        "Based on the following short story, describe the main setting or environment (place, time of day, atmosphere, surroundings and animation style) in one vivid sentence. "
        "Do NOT mention the main character here. Output only the description, no extra text.\n\n"
        f"Story:\n{story_script}\n"
    )
    env_resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": env_prompt}],
        max_tokens=60,
        temperature=0.7,
    )
    setting_description = env_resp.choices[0].message.content.strip().replace('"', '')

    print(f"Main character description: {main_character_description}")
    print(f"Supporting characters: {supporting_characters_description}")
    print(f"Setting/environment: {setting_description}")

    return story_script, main_character_description, supporting_characters_description, setting_description

STORY_SCRIPT, MAIN_CHARACTER_DESC, SUPP_CHAR_DESC, SETTING_DESC = generate_short_story(STORY_SEED)


3️⃣ Generate Relevant Images with DALL·E 3 (Segmented for Story)

In [ ]:
# ───────────────────────────────────────────────────────────────
# 3️⃣ Generate relevant images for story segments using DALL·E 3, ensuring visual consistency

import math
from PIL import Image, ImageOps
import requests
from openai import OpenAI
import re

def pad_to_vertical(img_path, target_size=(1080, 1920), fill_color=(0, 0, 0)):
    img = Image.open(img_path)
    img = ImageOps.contain(img, target_size, method=Image.LANCZOS)
    pad_img = Image.new("RGB", target_size, fill_color)
    offset_x = (target_size[0] - img.width) // 2
    offset_y = (target_size[1] - img.height) // 2
    pad_img.paste(img, (offset_x, offset_y))
    pad_img.save(img_path)
    return img_path

def split_script_for_images(script, num_images=7):
    sentences = re.split(r'(?<=[.!?]) +', script)
    avg = math.ceil(len(sentences) / num_images)
    segments = [' '.join(sentences[i:i + avg]) for i in range(0, len(sentences), avg)]
    return segments[:num_images]

def get_visual_prompt(client, segment, main_character_description, supporting_characters_description, setting_description):
    prompt = (
        "You are creating a DALL·E 3 prompt for a cinematic illustration. "
        "For the following story segment, pick the most visually engaging scene and choose the best shot type (wide, medium, or close-up) to convey the drama. "
        f"The main character: {main_character_description}. "
        f"Supporting characters: {supporting_characters_description}. "
        f"Setting/environment: {setting_description}. "
        "The image should depict the main character and (if present) supporting character(s) interacting within the environment, performing an action, dialogue, or displaying emotion relevant to the story segment. "
        "Make the composition consistent with the same people and setting throughout the sequence, even if the angle changes. "
        "Be visually vivid and concise (max 25 words). "
        f"Story segment: {segment}"
    )
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert at crafting prompts for cinematic storybook illustrations."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=120,
        temperature=0.85,
    )
    return response.choices[0].message.content.strip().strip('"')

def generate_dalle3_images(story_script, main_character_description, supporting_characters_description, setting_description, num_images=7):
    client = OpenAI()
    segments = split_script_for_images(story_script, num_images)
    image_files = []
    for i, segment in enumerate(segments):
        visual_prompt = get_visual_prompt(client, segment, main_character_description, supporting_characters_description, setting_description)
        dalle_prompt = f"Cinematic illustration, {visual_prompt}"
        print(f"🖼️ Prompt for DALL·E image {i + 1}: {dalle_prompt}")
        try:
            response = client.images.generate(
                model="dall-e-3",
                prompt=dalle_prompt,
                n=1,
                size="1024x1024",
                quality="standard",
                response_format="url"
            )
            img_url = response.data[0].url
            img_data = requests.get(img_url).content
            img_path = f"story_img_{i + 1}.png"
            with open(img_path, "wb") as f:
                f.write(img_data)
            pad_to_vertical(img_path)
            image_files.append(img_path)
            print(f"Generated image for segment {i + 1}: {img_path}")
        except Exception as e:
            print(f"❌ DALL·E image generation failed for segment {i + 1}: {e}")
            continue
    return image_files, segments

IMAGE_FILES, STORY_SEGMENTS = generate_dalle3_images(
    STORY_SCRIPT, MAIN_CHARACTER_DESC, SUPP_CHAR_DESC, SETTING_DESC, num_images=7
)


4️⃣ Generate Narration with OpenAI TTS (Echo Voice)

In [ ]:
# ───────────────────────────────────────────────────────────────
# 4️⃣ Generate TTS narration using OpenAI tts-1 (random dramatic voice, NEW API)

import random
from openai import OpenAI

def generate_tts_narration(story_script, output_path="narration.mp3"):
    client = OpenAI()  # Uses API key from environment

    # Choose a random dramatic voice
    voices = ["fable", "onyx", "echo"]
    selected_voice = random.choice(voices)
    print(f"🎤 Selected TTS voice: {selected_voice}")

    tts_response = client.audio.speech.create(
        model="tts-1",
        voice=selected_voice,
        input=story_script,
        response_format="mp3"
    )
    tts_response.stream_to_file(output_path)
    print(f"Narration audio saved to {output_path}")
    return output_path

NARRATION_AUDIO_PATH = generate_tts_narration(STORY_SCRIPT)


5️⃣ Generate Synced Subtitles (GPT for Chunks Matching Narration Timing)

In [ ]:
# ───────────────────────────────────────────────────────────────
# 5️⃣ Generate subtitle chunks using GPT and sync to narration (NEW API)

from openai import OpenAI

from aeneas.executetask import ExecuteTask
from aeneas.task import Task
import tempfile
import os

def split_text_into_max_lines(text, max_width_chars=25, max_lines=4):
    """
    Split text into multiple chunks where each chunk is max_lines (using word wrapping).
    Returns a list of lines chunks (each a string).
    """
    import textwrap
    wrapper = textwrap.TextWrapper(
        width=max_width_chars,
        break_long_words=True,
        break_on_hyphens=True
    )
    # Wrap full text to lines
    lines = wrapper.wrap(text)
    # Now group lines into max_lines per chunk
    chunks = []
    for i in range(0, len(lines), max_lines):
        chunk = "\n".join(lines[i:i + max_lines])
        chunks.append(chunk)
    return chunks

def chunk_script_for_subtitles(story_script, narration_path, max_width_chars=25, max_lines=4):
    """
    Generate subtitle chunks with precise timing using forced alignment.
    Now also guarantees that no subtitle chunk ever exceeds max_lines when word-wrapped.
    """
    import json
    import textwrap

    # 1. Create transcript file for aeneas
    with tempfile.NamedTemporaryFile(mode='w+', delete=False, suffix='.txt') as tf:
        tf.write(story_script)
        transcript_path = tf.name

    # 2. Configure and run aeneas forced alignment
    config_string = u"task_language=eng|is_text_type=plain|os_task_file_format=json"
    task = Task(config_string=config_string)
    task.audio_file_path_absolute = narration_path
    task.text_file_path_absolute = transcript_path
    task.sync_map_file_path_absolute = transcript_path + ".json"

    ExecuteTask(task).execute()
    task.output_sync_map_file()

    # 3. Parse the sync map and do post-processing split
    with open(task.sync_map_file_path_absolute, 'r') as f:
        sync_map = json.load(f)

    subtitle_timings = []
    for fragment in sync_map["fragments"]:
        text = fragment["lines"][0].strip()
        start = float(fragment["begin"])
        end = float(fragment["end"])
        duration = end - start

        # Split long lines to 4-lines-or-less chunks
        chunks = split_text_into_max_lines(text, max_width_chars=max_width_chars, max_lines=max_lines)
        chunks = [chunk for chunk in chunks if chunk.strip()]  # filter out empty/whitespace
        if not chunks:
            continue  # skip fragments with no text
        elif len(chunks) == 1:
            subtitle_timings.append({"text": chunks[0], "start": start, "end": end})
        else:
            chunk_duration = duration / len(chunks)
            for i, chunk in enumerate(chunks):
                chunk_start = start + i * chunk_duration
                chunk_end = chunk_start + chunk_duration
                subtitle_timings.append({"text": chunk, "start": chunk_start, "end": chunk_end})

    # 4. Clean up temp files
    os.remove(transcript_path)
    os.remove(task.sync_map_file_path_absolute)

    return subtitle_timings


# Install mutagen for MP3 duration reading if needed
!pip install mutagen --quiet

SUBTITLES = chunk_script_for_subtitles(STORY_SCRIPT, NARRATION_AUDIO_PATH)


6️⃣ Combine All Into Shorts Video (Images + Narration + Subtitles)

In [ ]:
# ───────────────────────────────────────────────────────────────
import sys

if sys.platform.startswith("linux"):
    TEXT_FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
elif sys.platform.startswith("win"):
    TEXT_FONT_PATH = "C:/Windows/Fonts/DejaVuSans-Bold.ttf"
else:
    TEXT_FONT_PATH = "DejaVuSans-Bold.ttf"
TEXT_FONTSIZE = 52

def _set_duration(clip, duration):
    if hasattr(clip, "with_duration"): return clip.with_duration(duration)
    return clip.set_duration(duration)

def _set_audio(clip, audio):
    if hasattr(clip, "with_audio"): return clip.with_audio(audio)
    return clip.set_audio(audio)

def _set_position(clip, pos):
    if hasattr(clip, "with_position"): return clip.with_position(pos)
    return clip.set_position(pos)

def _set_start(clip, start):
    if hasattr(clip, "with_start"): return clip.with_start(start)
    return clip.set_start(start)

def resize_fx(clip, newsize):
    if hasattr(clip, "resized"):
        return clip.resized(newsize)
    return clip.resize(newsize)

from moviepy import ImageClip, TextClip, CompositeVideoClip, concatenate_videoclips, AudioFileClip
import textwrap

def wrap_text_for_width(text, max_width_chars=25, max_lines=4):
    """Wrap text to fit within video frame width and limit to max_lines."""
    wrapper = textwrap.TextWrapper(
        width=max_width_chars,
        break_long_words=True,
        break_on_hyphens=True,
        max_lines=max_lines,
        placeholder=' [...]'
    )
    return "\n".join(wrapper.wrap(text))

def robust_textclip(text, start, end, font_path, fontsize):
    """
    Create a TextClip for subtitles, ensuring consistent formatting.
    """
    duration = end - start
    txt = None

    # Wrap text to ensure it fits within frame and limits to max lines
    wrapped_text = wrap_text_for_width(text, max_width_chars=25, max_lines=4)

    # Try caption mode with proper size constraints
    try:
        txt = TextClip(
            wrapped_text,
            font_size=fontsize,
            color='white',
            method='caption',
            size=(900, None)  # Max width for 1080px frame with margins
        )
        return txt
    except Exception as e:
        print(f"[Subtitle/caption failover] {e}")

    # Try label mode with font, fontsize
    try:
        txt = TextClip(
            wrapped_text,
            fontsize=fontsize,
            color='white',
            font=font_path,
            method='label'
        )
        return txt
    except Exception as e:
        print(f"[Subtitle/label failover] {e}")

    # Try plain text, no font specified
    try:
        txt = TextClip(wrapped_text)
        return txt
    except Exception as e:
        print(f"[Subtitle/plain failover] {e}")

    # Last resort: Render text with Pillow, use ImageClip
    try:
        from PIL import Image, ImageDraw, ImageFont
        import numpy as np

        W, H = 900, 200  # Subtitle box (width, height)
        bg_color = (0, 0, 0, 90)  # Semi-transparent background
        fg_color = (255, 255, 255, 255)

        # Load font, fallback to default PIL font if error
        try:
            font = ImageFont.truetype(font_path, fontsize)
        except Exception as e:
            print(f"[Subtitle/Pillow] Can't load '{font_path}', using default font: {e}")
            font = ImageFont.load_default()

        # Use the pre-wrapped text
        final_text = wrapped_text

        # Dummy image for measuring
        dummy_img = Image.new("RGBA", (W, H))
        draw = ImageDraw.Draw(dummy_img)
        bbox = draw.textbbox((0, 0), final_text, font=font)
        text_width, text_height = bbox[2] - bbox[0], bbox[3] - bbox[1]

        # Ensure text fits within our box width
        if text_width > W - 40:  # 40px total margin
            # If still too wide, try smaller font or more aggressive wrapping
            scale_factor = (W - 40) / text_width
            adjusted_fontsize = int(fontsize * scale_factor)
            try:
                font = ImageFont.truetype(font_path, adjusted_fontsize)
            except:
                font = ImageFont.load_default()

        # Now render actual image with proper dimensions
        margin_px = 60
        img_height = max(H, text_height + 40 + margin_px)
        img = Image.new('RGBA', (W, img_height), bg_color)
        draw = ImageDraw.Draw(img)

        # Center the text horizontally and vertically within the box
        text_x = (W - text_width) // 2
        text_y = (img_height - text_height - margin_px) // 2

        draw.text(
            (text_x, text_y),
            final_text,
            font=font,
            fill=fg_color,
            align='center'
        )

        np_img = np.array(img)
        txt = ImageClip(np_img, duration=duration)
        return txt
    except Exception as e:
        print(f"[Subtitle/Pillow ultimate fail] {e}")
        raise RuntimeError("Subtitle TextClip/ImageClip creation failed with all methods.")

def create_shorts_video(image_files, narration_path, subtitles, output_path="shorts_final.mp4"):
    audio = AudioFileClip(narration_path)
    total_duration = audio.duration
    num_images = len(image_files)
    img_duration = total_duration / num_images

    # Prepare image clips (robust resize)
    clips = []
    for idx, img in enumerate(image_files):
        base_clip = ImageClip(img)
        base_clip = resize_fx(base_clip, (1080, 1920))
        base_clip = _set_duration(base_clip, img_duration)
        base_clip = _set_position(base_clip, 'center')
        clips.append(base_clip)
    video = concatenate_videoclips(clips, method="compose")
    video = _set_audio(video, audio)

    # Prepare subtitle clips (with fade-in/out for nicer effect)
    subtitle_clips = []
    for sub in subtitles:
        txt = robust_textclip(
            sub["text"],
            sub["start"],
            sub["end"],
            TEXT_FONT_PATH,
            TEXT_FONTSIZE
        )
        txt = _set_start(txt, sub["start"])
        txt = _set_duration(txt, sub["end"] - sub["start"])
        txt = _set_position(txt, ("center", 120))
        # Only call margin if TextClip (not ImageClip)
        if isinstance(txt, TextClip):
            txt = txt.margin(top=10, opacity=0)
        subtitle_clips.append(txt)

    # Overlay subtitles
    final = CompositeVideoClip([video] + subtitle_clips)
    final.write_videofile(output_path, fps=24, codec="libx264", audio_codec="aac")
    print(f"Final shorts video saved to {output_path}")
    return output_path

SHORTS_VIDEO_PATH = create_shorts_video(IMAGE_FILES, NARRATION_AUDIO_PATH, SUBTITLES)

#@📝 8. Generate Metadata

In [ ]:
# 8️⃣ Generate Metadata (Story-based, clickworthy, Shorts-optimized)
def parse_metadata_response(text):
    title, desc, tags = None, None, []
    for line in text.splitlines():
        low = line.lower()
        if low.startswith("title:"):
            title = line.split(":",1)[1].strip().strip(' "\'')
            if title and not title[-1] in ('!', '?', '.'):
                title += '!'
        if low.startswith("description:"):
            desc = line.split(":",1)[1].strip()
            if desc:
                desc = desc
        if low.startswith("tags:"):
            tags = [t.strip() for t in line.split(":",1)[1].split(",")]
    required_tags = ["#shorts", "#short","#Shortsviral", "fyp", "viralshorts", "viralnews", "globalnews", "#trending", "#youtubeshorts"]
    tags = [tag for tag in tags if tag]
    for req_tag in required_tags:
        if req_tag.lower() not in [t.lower() for t in tags]:
            tags.append(req_tag)
    tags = [tag[:25] for tag in tags if tag]
    tags = list(set(tags))[:30]
    return title, desc, tags

def generate_metadata_from_story(story_text):
    prompt = (
        f"You are an expert at viral YouTube Shorts. Given the following short story, create:\n"
        "1. A clickworthy, emotional TITLE (<85 chars) based on the most powerful moment, surprise, or twist (NO bland summaries, NO boring names, must create curiosity!) and include 2-3 relevant hashtags\n"
        "2. A DESCRIPTION (<3500 characters) of the story, opening with a main keyword, includes a call to action (Like/Comment/Share), ends with a question. Then add 20-30 relevant hashtags at the end.\n"
        "3. Around 30 SEO TAGS (comma-separated), mixing story keywords, emotional triggers, setting, and all required viral tags (#shorts, #youtubeshorts, etc).\n"
        "\nStory:\n"
        f"{story_text}\n"
        "\nFormat:\n"
        "Title: ...\nDescription: ...\nTags: ...\n"
    )
    resp = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user","content":prompt}],
        temperature=0.8,
        max_tokens=3000
    )
    text = resp.choices[0].message.content.strip()
    return parse_metadata_response(text)

# Example usage (replace STORY_SCRIPT with your actual story variable)
title, description, tags = generate_metadata_from_story(STORY_SCRIPT)
print("📝 Metadata:\n ", title, "\n ", description, "\n ", tags)


📤 9. Upload to YouTube

In [ ]:
# ─── 9. Upload to YouTube ───────────────────────────────────────────────
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError
import time, os

def upload_video_to_youtube(
    file_path, title, description, tags,
    category_id=22, privacy="public",
    chunk_size=1024*1024, max_retries=5
):
    # Ensure tags is a list (not a comma-separated string)
    if isinstance(tags, str):
        tags = [t.strip() for t in tags.split(",") if t.strip()]

    # Enforce Shorts requirements
    if not any(t.lower() == "#shorts" for t in tags):
        tags.append("#shorts")

    body = {
        "snippet": {
            "title": title,
            "description": f"{description}\n\n#shorts",
            "tags": tags,
            "categoryId": str(category_id)
        },
        "status": {
            "privacyStatus": privacy,
            "selfDeclaredMadeForKids": False
        }
        # YouTube ignores contentDetails.duration in upload, but safe to keep.
    }

    media = MediaFileUpload(
        file_path,
        mimetype='video/mp4',
        chunksize=chunk_size,
        resumable=True
    )

    req = youtube.videos().insert(
        part="snippet,status",
        body=body,
        media_body=media
    )

    done = False
    retry = 0
    while not done:
        try:
            status, resp = req.next_chunk()
            if status:
                print(f"🟢 {int(status.progress()*100)}% uploaded")
            else:
                done = True
        except HttpError as e:
            if e.resp.status in [500,502,503,504] and retry < max_retries:
                retry += 1
                time.sleep(2**retry)
                print(f"⚠️ Retry {retry}")
            else:
                raise

    print("✅ Upload complete! Video ID:", resp['id'])
    return resp

# Usage with new metadata:
response = upload_video_to_youtube("shorts_final.mp4", title, description, tags)
print("🎉 Done! https://youtu.be/" + response["id"])
